#Name: Ankita Ghosh
#Reg number: 180905354
#Sec:A Roll number:41

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [ ]:
!nvcc --version

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

##Question 1

In [ ]:
%%cu
#include<stdio.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

const int n = 260;

__global__ void vec_add_blocks(int* a, int* b, int* c)
{
	int tid=blockIdx.x;
  if(tid<n){
      c[tid]=a[tid]+b[tid];
  }
}
__global__ void vec_add_threads(int* a, int* b, int* c)
{
	int tid=threadIdx.x;
  if(tid<n){
      c[tid]=a[tid]+b[tid];
  }
}
__global__ void vec_add_blocks_threads(int* a, int* b, int* c)
{
	int tid=threadIdx.x + blockIdx.x*blockDim.x;
  while ( tid < n ){
      c[tid]=a[tid]+b[tid];
      tid+= blockDim.x * gridDim.x ;
  }
}

int main()
{

	int a[n],b[n],c[n];
	int *d_a,*d_b,*d_c;
	int size=sizeof(int);

	cudaMalloc((void **)&d_a,size*n);
  cudaMalloc((void **)&d_b,size*n);
  cudaMalloc((void **)&d_c,size*n);
	
  for(int i=0;i<n;i++){
      a[i]=i;
      b[i]=i+10;
  }
  printf("Array A: ");
   for(int i=0;i<n;i++){
      printf("%d ",a[i]);
  }
  printf("\n");
   
   printf("Array B: ");
   for(int i=0;i<n;i++){
      printf("%d ",b[i]);
  }
  printf("\n");
  printf("\n");

	cudaMemcpy(d_a,&a,size*n,cudaMemcpyHostToDevice);
  cudaMemcpy(d_b,&b,size*n,cudaMemcpyHostToDevice);
	vec_add_blocks<<<n,1>>>(d_a,d_b,d_c);
	cudaMemcpy(&c,d_c,size*n,cudaMemcpyDeviceToHost);
  printf("Using blocks: ");
  for(int i=0;i<n;i++){
      printf("%d ",c[i]);
  }
  printf("\n");
  vec_add_threads<<<1,n>>>(d_a,d_b,d_c);
	cudaMemcpy(&c,d_c,size*n,cudaMemcpyDeviceToHost);
  printf("Using n threads:");
  for(int i=0;i<n;i++){
      printf("%d ",c[i]);
  }
  printf("\n");
  vec_add_blocks_threads<<<(n+255)/256,256>>>(d_a,d_b,d_c);
	cudaMemcpy(&c,d_c,size*n,cudaMemcpyDeviceToHost);
  printf("Varying blocks and 256 threads:");
  for(int i=0;i<n;i++){
      printf("%d ",c[i]);
  }
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);
}

Array A: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 
Array B: 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 

##Question 2

In [ ]:
%%cu
#include<stdio.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

const int n=10;

__global__ void selection_sort_parallel(int* a, int* c)
{
	int tid=threadIdx.x;
  int pos=0;
  if(tid<n){
      for(int j=0;j<n;j++){
          if((a[j]<a[tid])||(a[j]==a[tid])&&(j<tid)){
              pos=pos+1;
          }
      }
      c[pos]=a[tid];
  }
}

int main()
{

	int c[n];
	int *d_a,*d_c;
	int size=sizeof(int);
  int a[10]={2,24,5,67,9,100,234,12,56,1234};
  printf("Original array: ");
  for(int i=0;i<n;i++)
  {
      printf("%d ",a[i]);
  }
 printf("\n");
	cudaMalloc((void **)&d_a,size*n);
  cudaMalloc((void **)&d_c,size*n);
	cudaMemcpy(d_a,&a,size*n,cudaMemcpyHostToDevice);
  cudaMemcpy(d_c,&c,size*n,cudaMemcpyHostToDevice);
	selection_sort_parallel<<<1,n>>>(d_a,d_c);
	cudaMemcpy(&c,d_c,size*n,cudaMemcpyDeviceToHost);
  printf("Sorted array: ");
  for(int i=0;i<n;i++){
      printf("%d ",c[i]);
  }
  cudaFree(d_a);
  cudaFree(d_c);
}

Original array: 2 24 5 67 9 100 234 12 56 1234 
Sorted array: 2 5 9 12 24 56 67 100 234 1234 


##Question 3

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
__global__ void oddsort(int *a,int n)
{
    
    int i=threadIdx.x+blockDim.x*blockIdx.x;
    int el1=2*i+1;int el2=2*i+2;
    if(el1<n && el2<n)
    {
        if(a[el1]>a[el2])
        {
            int temp=a[el1];
            a[el1]=a[el2];
            a[el2]=temp;
        }
    }
}
__global__ void evensort(int *a,int n)
{
    int i=threadIdx.x+blockDim.x*blockIdx.x;
    int el1=2*i;int el2=2*i+1;
    if(el1<n && el2<n)
    {
        if(a[el1]>a[el2])
        {
            int temp=a[el1];
            a[el1]=a[el2];
            a[el2]=temp;
        }
    }
}
void hostfunc(int *a,int n)
{
    int size = n*sizeof(int);
    int *d_a;
    cudaMalloc((void**)&d_a,size);
    cudaMemcpy(d_a,a,size,cudaMemcpyHostToDevice);
    for(int i=0;i<n/2;i++)
    {
        evensort<<<1,256>>>(d_a,n);
        oddsort<<<1,256>>>(d_a,n);
        
    }
    cudaMemcpy(a,d_a,size,cudaMemcpyDeviceToHost);
    cudaFree(d_a);
}
int main()
{
    int n=10;
    int a[10]={2,24,5,67,9,100,234,12,56,1234};
    printf("Original array: ");
     for(int i=0;i<n;i++)
    {
        printf("%d ",a[i]);
    }
    printf("\n");

    
    hostfunc(a,n);
   printf("Sorted array: ");
    for(int i=0;i<n;i++)
    {
        printf("%d ",a[i]);
    }
    printf("\n");
}

Original array: 2 24 5 67 9 100 234 12 56 1234 
Sorted array: 2 5 9 12 24 56 67 100 234 1234 

